In [37]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import time 
from tensorflow.keras.models import Sequential, load_model
from keras.layers import SimpleRNN, Dense
import tensorflow as tf
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [38]:
data_path_and_labels = pd.read_csv("../mapped_data/xalan-2.6.csv")
data_path_and_labels['data_path'][0]

'dataset/promise/defect data/xalan-2.6/xalan-j_2_6_0/src/org/w3c/dom/xpath/XPathNamespace.java.restrictedcontent.embed'

In [39]:
data_path_and_labels['data_path'] = '../' + data_path_and_labels['data_path']
data_path_and_labels['data_path'][0]

'../dataset/promise/defect data/xalan-2.6/xalan-j_2_6_0/src/org/w3c/dom/xpath/XPathNamespace.java.restrictedcontent.embed'

In [40]:
data_path_and_labels['data_path'] = data_path_and_labels['data_path'].str.replace('.embed','')
data_path_and_labels['data_path'][0]

'../dataset/promise/defect data/xalan-2.6/xalan-j_2_6_0/src/org/w3c/dom/xpath/XPathNamespace.java.restrictedcontent'

In [41]:
words = []
for index, row in data_path_and_labels.iterrows():
    with open(row['data_path']) as f:
        lines = f.read().splitlines()
    words.append(lines)

In [42]:
d = pd.read_csv("../mapped_data/xalan-2.6.csv")
d['data_path'][0]

'dataset/promise/defect data/xalan-2.6/xalan-j_2_6_0/src/org/w3c/dom/xpath/XPathNamespace.java.restrictedcontent.embed'

In [43]:
d['data_path'] = '../' + d['data_path']
d['data_path'][0]

'../dataset/promise/defect data/xalan-2.6/xalan-j_2_6_0/src/org/w3c/dom/xpath/XPathNamespace.java.restrictedcontent.embed'

In [44]:
d

,data_path,bugs
0,../dataset/promise/defect data/xalan-2.6/xalan...,1
1,../dataset/promise/defect data/xalan-2.6/xalan...,1
2,../dataset/promise/defect data/xalan-2.6/xalan...,1
3,../dataset/promise/defect data/xalan-2.6/xalan...,1
4,../dataset/promise/defect data/xalan-2.6/xalan...,1
...,...,...
870,../dataset/promise/defect data/xalan-2.6/xalan...,1
871,../dataset/promise/defect data/xalan-2.6/xalan...,1
872,../dataset/promise/defect data/xalan-2.6/xalan...,1
873,../dataset/promise/defect data/xalan-2.6/xalan...,1


In [45]:
train, test = train_test_split(d, test_size=0.2)

In [46]:
test_x = []
for index, row in test.iterrows():
    with open(row['data_path']) as f:
        lines = list(map(float, f.read().splitlines())) 
    test_x.append(lines)

In [47]:
train_x = []
for index, row in train.iterrows():
    with open(row['data_path']) as f:
        lines = list(map(float, f.read().splitlines())) 
    train_x.append(lines)

In [48]:
vocab = float(max(np.max(train_x), np.max(test_x)))
vocab

9675.0

In [49]:
train_y = train[' bugs'].to_numpy()
test_y = test[' bugs'].to_numpy()

In [50]:
train_y

array([1, 0, 1, 2, 2, 0, 2, 0, 0, 1, 1, 1, 4, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 2, 2, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 2, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 5, 1, 0, 2, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 2, 0, 2,
       1, 0, 0, 1, 1, 1, 1, 0, 2, 3, 1, 0, 2, 0, 0, 2, 1, 3, 1, 1, 0, 0,
       5, 0, 2, 3, 1, 1, 2, 0, 0, 1, 1, 5, 1, 1, 1, 1, 0, 1, 2, 2, 1, 1,
       0, 0, 2, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 1, 2, 1, 6, 0, 0, 1, 0, 0, 0, 1, 2, 1, 0, 3, 0, 0, 2, 1,
       0, 0, 0, 2, 0, 0, 0, 0, 1, 1, 0, 0, 0, 3, 0, 0, 0, 3, 2, 0, 5, 0,
       1, 0, 2, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 2, 0, 2, 0, 2, 0, 0,
       0, 0, 4, 0, 1, 1, 0, 1, 1, 0, 0, 1, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 2, 0, 1, 0, 2, 1, 0, 2, 1, 3, 0, 3, 1, 2, 1, 1, 1, 0, 0, 1,
       0, 1, 0, 1, 1, 0, 2, 1, 0, 3, 1, 0, 0, 1, 0,

In [51]:
train_x = np.array(train_x)
test_x = np.array(test_x)

In [52]:
train_x.shape

(700, 7389)

In [53]:
# define the model
model = Sequential()
model.add(tf.keras.layers.Embedding(10671, 30, input_length=train_x.shape[1]))
model.add(tf.keras.layers.Flatten())
model.add(Dense(1, activation='relu'))
# compile the model
model.compile(loss = 'mse' , optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [54]:
# Fitting the model on training data.
import datetime
log_dir = '/home/bavanya/Desktop/tensorboard/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history = model.fit(train_x, train_y, epochs = 100, callbacks=[tensorboard_callback])

Epoch 1/100
22/22 [==============================] - 1s 35ms/step - loss: 1.1689 - mse: 1.1689 - mae: 0.7882 - root_mean_squared_error: 1.0812 - mean_squared_logarithmic_error: 0.2798
Epoch 2/100
22/22 [==============================] - 1s 25ms/step - loss: 0.9313 - mse: 0.9313 - mae: 0.7040 - root_mean_squared_error: 0.9650 - mean_squared_logarithmic_error: 0.2153
Epoch 3/100
22/22 [==============================] - 1s 24ms/step - loss: 0.8310 - mse: 0.8310 - mae: 0.6719 - root_mean_squared_error: 0.9116 - mean_squared_logarithmic_error: 0.1955
Epoch 4/100
22/22 [==============================] - 1s 24ms/step - loss: 0.7480 - mse: 0.7480 - mae: 0.6472 - root_mean_squared_error: 0.8649 - mean_squared_logarithmic_error: 0.1815
Epoch 5/100
22/22 [==============================] - 1s 23ms/step - loss: 0.6970 - mse: 0.6970 - mae: 0.6258 - root_mean_squared_error: 0.8349 - mean_squared_logarithmic_error: 0.1729
Epoch 6/100
22/22 [==============================] - 1s 24ms/step - loss: 0.6785

22/22 [==============================] - 1s 24ms/step - loss: 0.0656 - mse: 0.0656 - mae: 0.1488 - root_mean_squared_error: 0.2560 - mean_squared_logarithmic_error: 0.0226
Epoch 45/100
22/22 [==============================] - 1s 26ms/step - loss: 0.0614 - mse: 0.0614 - mae: 0.1441 - root_mean_squared_error: 0.2479 - mean_squared_logarithmic_error: 0.0216
Epoch 46/100
22/22 [==============================] - 1s 25ms/step - loss: 0.0604 - mse: 0.0604 - mae: 0.1441 - root_mean_squared_error: 0.2457 - mean_squared_logarithmic_error: 0.0216
Epoch 47/100
22/22 [==============================] - 1s 24ms/step - loss: 0.0551 - mse: 0.0551 - mae: 0.1309 - root_mean_squared_error: 0.2346 - mean_squared_logarithmic_error: 0.0191
Epoch 48/100
22/22 [==============================] - 1s 25ms/step - loss: 0.0523 - mse: 0.0523 - mae: 0.1242 - root_mean_squared_error: 0.2286 - mean_squared_logarithmic_error: 0.0182
Epoch 49/100
22/22 [==============================] - 1s 26ms/step - loss: 0.0507 - mse:

22/22 [==============================] - 1s 23ms/step - loss: 0.0158 - mse: 0.0158 - mae: 0.0486 - root_mean_squared_error: 0.1258 - mean_squared_logarithmic_error: 0.0055
Epoch 89/100
22/22 [==============================] - 1s 24ms/step - loss: 0.0149 - mse: 0.0149 - mae: 0.0468 - root_mean_squared_error: 0.1219 - mean_squared_logarithmic_error: 0.0053
Epoch 90/100
22/22 [==============================] - 1s 27ms/step - loss: 0.0143 - mse: 0.0143 - mae: 0.0419 - root_mean_squared_error: 0.1197 - mean_squared_logarithmic_error: 0.0052
Epoch 91/100
22/22 [==============================] - 1s 24ms/step - loss: 0.0144 - mse: 0.0144 - mae: 0.0450 - root_mean_squared_error: 0.1201 - mean_squared_logarithmic_error: 0.0050
Epoch 92/100
22/22 [==============================] - 1s 24ms/step - loss: 0.0136 - mse: 0.0136 - mae: 0.0417 - root_mean_squared_error: 0.1166 - mean_squared_logarithmic_error: 0.0048
Epoch 93/100
22/22 [==============================] - 1s 24ms/step - loss: 0.0136 - mse:

In [55]:
%tensorboard --logdir '/home/bavanya/Desktop/tensorboard/'

Reusing TensorBoard on port 6006 (pid 21467), started 2:19:41 ago. (Use '!kill 21467' to kill it.)

In [56]:
score = model.evaluate(test_x, test_y)
score

6/6 [==============================] - 0s 13ms/step - loss: 0.4099 - mse: 0.4099 - mae: 0.4132 - root_mean_squared_error: 0.6403 - mean_squared_logarithmic_error: 0.1197


[0.4099401831626892,
 0.4099401831626892,
 0.41315197944641113,
 0.640265703201294,
 0.11968965828418732]

In [21]:
data_path_and_labels = pd.read_csv("../mapped_data/xalan-2.6.csv")
data_path_and_labels['data_path'][0]

'dataset/promise/defect data/xalan-2.6/xalan-j_2_6_0/src/org/w3c/dom/xpath/XPathNamespace.java.restrictedcontent.embed'

In [22]:
data_path_and_labels['data_path'] = '../' + data_path_and_labels['data_path']
data_path_and_labels['data_path'][0]

'../dataset/promise/defect data/xalan-2.6/xalan-j_2_6_0/src/org/w3c/dom/xpath/XPathNamespace.java.restrictedcontent.embed'

In [23]:
embeddings = []
for index, row in data_path_and_labels.iterrows():
    with open(row['data_path']) as f:
        lines = list(map(float, f.read().splitlines())) 
    embeddings.append(lines)

In [24]:
df = pd.DataFrame(embeddings)
df

,0,1,2,3,4,5,6,7,8,9,...,7379,7380,7381,7382,7383,7384,7385,7386,7387,7388
0,2656.0,8772.0,2642.0,5294.0,1051.0,1758.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2646.0,3939.0,2649.0,4500.0,2302.0,8279.0,2654.0,3948.0,2654.0,7547.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2660.0,8772.0,32.0,8772.0,1730.0,8772.0,2146.0,8772.0,160.0,8772.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2649.0,4423.0,1838.0,3868.0,1758.0,9286.0,8772.0,8285.0,1838.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2654.0,6290.0,2302.0,7981.0,2302.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
870,5844.0,1597.0,5844.0,1601.0,5844.0,1599.0,5844.0,1598.0,1597.0,1601.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
871,5844.0,1597.0,5844.0,1601.0,5844.0,1599.0,5844.0,1598.0,1597.0,1601.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
872,5844.0,1597.0,5844.0,1601.0,5844.0,1599.0,5844.0,1598.0,1597.0,1601.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
873,5844.0,1597.0,5844.0,1601.0,5844.0,1599.0,5844.0,1598.0,1597.0,1601.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
data = pd.concat([data_path_and_labels, df], axis = 1)
data = data.drop('data_path', 1)
data

,bugs,0,1,2,3,4,5,6,7,8,...,7379,7380,7381,7382,7383,7384,7385,7386,7387,7388
0,1,2656.0,8772.0,2642.0,5294.0,1051.0,1758.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2646.0,3939.0,2649.0,4500.0,2302.0,8279.0,2654.0,3948.0,2654.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,2660.0,8772.0,32.0,8772.0,1730.0,8772.0,2146.0,8772.0,160.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,2649.0,4423.0,1838.0,3868.0,1758.0,9286.0,8772.0,8285.0,1838.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,2654.0,6290.0,2302.0,7981.0,2302.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
870,1,5844.0,1597.0,5844.0,1601.0,5844.0,1599.0,5844.0,1598.0,1597.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
871,1,5844.0,1597.0,5844.0,1601.0,5844.0,1599.0,5844.0,1598.0,1597.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
872,1,5844.0,1597.0,5844.0,1601.0,5844.0,1599.0,5844.0,1598.0,1597.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
873,1,5844.0,1597.0,5844.0,1601.0,5844.0,1599.0,5844.0,1598.0,1597.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
train, test = train_test_split(data, test_size=0.2)

In [27]:
train

,bugs,0,1,2,3,4,5,6,7,8,...,7379,7380,7381,7382,7383,7384,7385,7386,7387,7388
7,2,1014.0,5544.0,5844.0,8302.0,514.0,5255.0,2302.0,8302.0,516.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
442,0,2256.0,1838.0,8409.0,2256.0,8868.0,9276.0,288.0,3601.0,4700.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
588,2,2297.0,5844.0,6456.0,2297.0,9483.0,5844.0,7374.0,2302.0,7357.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
459,1,5844.0,1597.0,5844.0,1601.0,5844.0,1599.0,5844.0,1598.0,1597.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
705,0,1840.0,2302.0,363.0,2302.0,2078.0,3573.0,343.0,1973.0,4558.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,2,2026.0,2526.0,6915.0,2526.0,6423.0,3573.0,6755.0,77.0,77.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
277,0,1538.0,1538.0,4694.0,1975.0,4694.0,5619.0,3573.0,8302.0,5608.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
55,1,1025.0,4466.0,9221.0,2442.0,2645.0,3864.0,5541.0,5693.0,2184.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
767,0,2062.0,1559.0,6805.0,2062.0,2062.0,6266.0,1559.0,6805.0,6266.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
test

,bugs,0,1,2,3,4,5,6,7,8,...,7379,7380,7381,7382,7383,7384,7385,7386,7387,7388
592,0,493.0,7706.0,2637.0,6169.0,2637.0,8315.0,2637.0,8302.0,2636.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
113,0,1083.0,4879.0,1758.0,4882.0,1764.0,5000.0,1071.0,9167.0,4223.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
150,0,397.0,5844.0,1126.0,5844.0,1718.0,5844.0,1722.0,5844.0,1999.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
754,0,1055.0,1055.0,1055.0,5101.0,434.0,8302.0,1055.0,7479.0,5844.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
729,1,438.0,438.0,438.0,7286.0,2302.0,7286.0,438.0,4263.0,2405.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,2,1072.0,1838.0,6621.0,2016.0,2015.0,335.0,1009.0,605.0,1403.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
617,0,178.0,178.0,178.0,7598.0,1941.0,7598.0,178.0,3828.0,306.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59,2,1043.0,3573.0,6941.0,8659.0,9368.0,3573.0,6941.0,9368.0,5334.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
297,0,1739.0,1082.0,3063.0,2452.0,3064.0,1758.0,1739.0,4694.0,1975.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
Y_train = train[' bugs']
Y_test = test[' bugs']

In [30]:
train_y = Y_train.to_numpy()
test_y = Y_test.to_numpy()

In [31]:
train = train.drop(' bugs', axis=1)
test = test.drop(' bugs', axis=1)

In [32]:
X_train = train.values
X_test = test.values

In [33]:
train_x = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
test_x = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [34]:
# Designing and initializing the model.
model = Sequential()
model.add(SimpleRNN(100, input_shape = (1,20), dropout = 0.2, return_sequences=True))
model.add(SimpleRNN(80, dropout = 0.2, return_sequences=True))
model.add(SimpleRNN(60, dropout = 0.2, return_sequences=False))
model.add(Dense(1, activation = 'relu'))
model.compile(loss = 'mse' , optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [35]:
# Fitting the model on training data.
history = model.fit(train_x, train_y, epochs = 100)

Epoch 1/100
22/22 [==============================] - 2s 8ms/step - loss: 1.5309 - mse: 1.5309 - mae: 0.7744 - root_mean_squared_error: 1.2373 - mean_squared_logarithmic_error: 0.3908
Epoch 2/100
22/22 [==============================] - 0s 7ms/step - loss: 1.4882 - mse: 1.4882 - mae: 0.7571 - root_mean_squared_error: 1.2199 - mean_squared_logarithmic_error: 0.3756
Epoch 3/100
22/22 [==============================] - 0s 7ms/step - loss: 1.5077 - mse: 1.5077 - mae: 0.7730 - root_mean_squared_error: 1.2279 - mean_squared_logarithmic_error: 0.3828
Epoch 4/100
22/22 [==============================] - 0s 7ms/step - loss: 1.4781 - mse: 1.4781 - mae: 0.7559 - root_mean_squared_error: 1.2158 - mean_squared_logarithmic_error: 0.3729
Epoch 5/100
22/22 [==============================] - 0s 7ms/step - loss: 1.5048 - mse: 1.5048 - mae: 0.7667 - root_mean_squared_error: 1.2267 - mean_squared_logarithmic_error: 0.3812
Epoch 6/100
22/22 [==============================] - 0s 8ms/step - loss: 1.5081 - mse

22/22 [==============================] - 0s 6ms/step - loss: 1.0487 - mse: 1.0487 - mae: 0.7011 - root_mean_squared_error: 1.0240 - mean_squared_logarithmic_error: 0.2529
Epoch 46/100
22/22 [==============================] - 0s 6ms/step - loss: 1.0055 - mse: 1.0055 - mae: 0.6956 - root_mean_squared_error: 1.0027 - mean_squared_logarithmic_error: 0.2486
Epoch 47/100
22/22 [==============================] - 0s 6ms/step - loss: 1.0105 - mse: 1.0105 - mae: 0.6917 - root_mean_squared_error: 1.0053 - mean_squared_logarithmic_error: 0.2475
Epoch 48/100
22/22 [==============================] - 0s 6ms/step - loss: 1.0740 - mse: 1.0740 - mae: 0.7146 - root_mean_squared_error: 1.0363 - mean_squared_logarithmic_error: 0.2616
Epoch 49/100
22/22 [==============================] - 0s 6ms/step - loss: 0.9865 - mse: 0.9865 - mae: 0.6882 - root_mean_squared_error: 0.9932 - mean_squared_logarithmic_error: 0.2433
Epoch 50/100
22/22 [==============================] - 0s 6ms/step - loss: 1.0438 - mse: 1.043

22/22 [==============================] - 0s 6ms/step - loss: 0.9646 - mse: 0.9646 - mae: 0.6709 - root_mean_squared_error: 0.9821 - mean_squared_logarithmic_error: 0.2369
Epoch 90/100
22/22 [==============================] - 0s 6ms/step - loss: 0.9552 - mse: 0.9552 - mae: 0.6791 - root_mean_squared_error: 0.9773 - mean_squared_logarithmic_error: 0.2392
Epoch 91/100
22/22 [==============================] - 0s 6ms/step - loss: 0.9109 - mse: 0.9109 - mae: 0.6728 - root_mean_squared_error: 0.9544 - mean_squared_logarithmic_error: 0.2329
Epoch 92/100
22/22 [==============================] - 0s 6ms/step - loss: 0.9310 - mse: 0.9310 - mae: 0.6543 - root_mean_squared_error: 0.9649 - mean_squared_logarithmic_error: 0.2306
Epoch 93/100
22/22 [==============================] - 0s 6ms/step - loss: 0.9041 - mse: 0.9041 - mae: 0.6612 - root_mean_squared_error: 0.9509 - mean_squared_logarithmic_error: 0.2265
Epoch 94/100
22/22 [==============================] - 0s 6ms/step - loss: 0.8953 - mse: 0.895

In [36]:
score = model.evaluate(test_x, test_y)
score

6/6 [==============================] - 0s 2ms/step - loss: 0.7463 - mse: 0.7463 - mae: 0.6014 - root_mean_squared_error: 0.8639 - mean_squared_logarithmic_error: 0.2062


[0.746295154094696,
 0.746295154094696,
 0.6014357209205627,
 0.8638837337493896,
 0.2061806172132492]